In [1]:
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
printint = np.vectorize(np.int)
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from pygwb.detector import Interferometer
from pygwb import network

# Analysis parameters

In [3]:
'''
Analysis parameters
this will become an ini file to pass to an argparser
'''
t0 = 1247644138 # start GPS time
tf = 1247645038 # end GPS time
data_type='public' # private -> running on LIGO data grid
channel_suffix = 'GWOSC-16KHZ_R1_STRAIN' # detector name will be added later
new_sample_rate = 4096 # sampled rate after resampling
cutoff_frequency = 11 # high pass filter cutoff frequency
segment_duration = 192 # also fftlength in pre-processing
number_cropped_seconds = 2 # no. of secs to crop after highpass and resampling (default = 2 sec)
window_downsampling = "hamming" # filter used for downsampling (default = 'hamming')
ftype = "fir" # filter type used for downsampling
window_fftgram = "hann" # window used for fft (used CSD and PSD estimation)
overlap = segment_duration/2 # overlapping between segments
frequency_resolution = 1.0/32 # final frequency resolution of CSD and PSD 
polarization = 'tensor' # for regular analysis 
alpha = 0 # power law index
fref = 25 # Hz
flow = 20 # Hz
fhigh = 1726 # Hz
coarse_grain = 0  # 0 - pwelch PSD estimate; 1 - corase-grain PSD estimate 
if coarse_grain:
    fft_length = segment_duration
else:
    fft_length = int(1/frequency_resolution)

# Create IFO objects

In [4]:
import bilby

In [5]:
ifo_H1 = Interferometer.get_empty_interferometer('H1')
ifo_L1 = Interferometer.get_empty_interferometer('L1')

# Get data and pre-process data

In [6]:
from pygwb import pre_processing

In [7]:
ifo_H1.set_pre_processed_timeseries_from_channel_name(
    ifo_H1.name,
    t0,
    tf,
    data_type,
    ifo_H1.name+':'+channel_suffix,
    new_sample_rate,
    cutoff_frequency,
    segment_duration,
    number_cropped_seconds=number_cropped_seconds,
    window_downsampling=window_downsampling,
    ftype=ftype,
)

ifo_L1.set_pre_processed_timeseries_from_channel_name(
    ifo_L1.name,
    t0,
    tf,
    data_type,
    ifo_L1.name+':'+channel_suffix,
    new_sample_rate,
    cutoff_frequency,
    segment_duration,
    number_cropped_seconds=number_cropped_seconds,
    window_downsampling=window_downsampling,
    ftype=ftype,
)

# initialise a baseline with the new ifos

In [8]:
from pygwb.baseline import Baseline

In [9]:
base_HL = Baseline('HL', ifo_H1, ifo_L1, duration=segment_duration, do_overlap=True)

/home/arianna.renzini/.conda/envs/AIRlab/lib/python3.8/site-packages/pygwb/baseline.py:171: UserWarning: baseline frequencies have not been set.
  warnings.warn("baseline frequencies have not been set.")


# Make PSDs and CSD

In [10]:
from pygwb import spectral
from pygwb.util import window_factors

In [11]:
# calculate PSD all possible segments for detector 1
base_HL.set_cross_and_power_spectral_density(frequency_resolution)
# base_HL.set_average_PSDs()

base_HL.set_average_power_spectral_densities()
base_HL.set_average_cross_spectral_density()

segment_starttime = base_HL.csd.times.value - (segment_duration / 2) # for later use

# initialise baseline - get ORF

In [49]:
freqs = base_HL.interferometer_1.average_psd.yindex.value
base_HL.set_frequencies(freqs)

In [51]:
deltaF = freqs[1]-freqs[0]
try:
    assert abs(deltaF - frequency_resolution) < 1e-6 # within machine (floating point) precision
except ValueError:  
    print('Frequency resolution in PSD/CSD is different than requested.')

# Band limiting

In [53]:
freq_band_cut = (freqs>=flow) & (freqs<=fhigh)
#naive_psd_1 = naive_psd_1.crop_frequencies(flow, fhigh+deltaF)
#naive_psd_2 = naive_psd_2.crop_frequencies(flow, fhigh+deltaF)
avg_psd_1 = base_HL.interferometer_1.average_psd.crop_frequencies(flow, fhigh+deltaF)
avg_psd_2 = base_HL.interferometer_2.average_psd.crop_frequencies(flow, fhigh+deltaF)
csd = base_HL.average_csd.crop_frequencies(flow, fhigh+deltaF)

orf = base_HL.overlap_reduction_function[freq_band_cut]

# calcualte Y and sigma

In [55]:
from pygwb.constants import H0
from pygwb.constants import speed_of_light
from pygwb.util import window_factors

# window factors
w1w2bar, w1w2squaredbar, _, _ = window_factors(4096*192)

#will be in postproc

def calculate_Yf_varf(freqs, csd, avg_psd_1, avg_psd_2, orf, fref, alpha,):    
    S_alpha = 3 * H0**2 / (10 * np.pi**2) / freqs**3 * (freqs/fref)**alpha
    Y_fs = np.real(csd)/(orf * S_alpha)
    var_fs = 1 / (2 * segment_duration * (freqs[1]-freqs[0])) * avg_psd_1 * avg_psd_2 / (orf**2 * S_alpha**2)
    var_fs = var_fs * w1w2squaredbar / w1w2bar ** 2
    return Y_fs, var_fs

In [56]:
# Create Y_f and sigma_f of stochastic analysis for every segment
freqs = freqs[freq_band_cut]
Y_fs, var_fs = calculate_Yf_varf(freqs, csd.value, 
                                   avg_psd_1.value, 
                                   avg_psd_2.value, 
                                   orf, fref, alpha)

(3, 54593) (3, 54593)


In [67]:
#notching
from scipy.io import loadmat
stochastic_mat = loadmat('/home/shivaraj.kandhasamy/stochastic/iso/misc/pyGWB/command/stochData_gwosc.mat',struct_as_record=False, squeeze_me=True)
notch_freq = np.real(stochastic_mat['ptEst_ff']==0)
Y_fs[:,notch_freq] = 0
var_fs[:,notch_freq] = np.Inf # some large number

# Make final point estimate and error bar

In [68]:
from pygwb.util import calc_bias
from pygwb.postprocessing import postprocess_Y_sigma
from pygwb.util import calc_Y_sigma_from_Yf_varf

In [69]:
Y_f_new, var_f_new = postprocess_Y_sigma(Y_fs, var_fs, segment_duration, deltaF, new_sample_rate, notch_freq)

/home/arianna.renzini/.conda/envs/AIRlab/lib/python3.8/site-packages/pygwb/postprocessing.py:548: RuntimeWarning: invalid value encountered in true_divide
  Y_f_new = (X_odd * (1- (k/2) * sigma2_oo * sigma2IJ) + X_even * (1- (k/2) * sigma2_ee * sigma2IJ))/ (
/home/arianna.renzini/.conda/envs/AIRlab/lib/python3.8/site-packages/pygwb/postprocessing.py:554: RuntimeWarning: divide by zero encountered in true_divide
  var_f_new = (1 / inv_var_f_new) * bias**2


In [70]:
Y_pyGWB_new, sigma_pyGWB_new = calc_Y_sigma_from_Yf_varf(Y_f_new, var_f_new, freqs, alpha, fref)
#Y_pyGWB_new_alt, sigma_pyGWB_new_alt = calc_Y_sigma_from_Yf_varf(Y_f_new_alt, var_f_new_alt, freqs, alpha, fref)

In [71]:
print('\tpyGWB: %e'%(Y_pyGWB_new))
print('\tpyGWB: %e'%(sigma_pyGWB_new))

	pyGWB: -2.969473e-06
	pyGWB: 1.228472e-06


In [72]:
# Calculate Y_f and sigma_f of stochastic analysis for each segment
for ii in range(0,3):
    Y_seg, var_seg = calculate_Yf_varf(freqs, csd[0].value, 
                                   avg_psd_1[ii].value, 
                                   avg_psd_2[ii].value, 
                                   orf, fref, alpha)
    Y_seg[notch_freq] = 0
    var_seg[notch_freq] = 1e80 # some large number
    Y_pyGWB_seg, sigma_pyGWB_seg = calc_Y_sigma_from_Yf_varf(Y_seg, var_seg, freqs, alpha, fref)
    print(sigma_pyGWB_seg/stochastic_mat['ccSigma'][ii]*192)

0.9976910482914612
0.9973249909630746
0.9973286745862678
